In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from IPython.display import display, display_html
from sklearn.dummy import DummyClassifier


## Acquire

In [ ]:
#Import Libraries
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import acquire

In [ ]:
# Acquire
# Create function to retrieve telco_churn data
telco = acquire.get_telco_churn_data()

   

In [ ]:
# Create a function that retrieves the necessary connection URL.

def get_connection(db_name):
    """
    This function uses my info from my env file to
    create a connection url to access the Codeup db.
    """
    return f'mysql+pymysql://{user}:{password}@{host}/{db_name}'


In [ ]:
# Create function to retrieve telco_churn data
def get_telco_churn_data():
    """
    This function reads in the Telco Churn data from the Codeup db
    and returns a pandas DataFrame with all columns.
    """
    filename = 'telco.csv'

    if os.path.isfile(filename):
        return pd.read_csv(filename)

    else:
        sql = '''
                SELECT *
                FROM customers
                JOIN payment_types USING (payment_type_id)
                JOIN contract_types USING (contract_type_id)
                JOIN internet_service_types USING (internet_service_type_id);
                '''

        df = pd.read_sql(sql, get_connection('telco_churn'))

        df.to_csv(filename, index=False)

        return df


_________

## Prepare

In [ ]:
#Import prepare functions
import prepare

In [ ]:
def prep_telco_churn(df):
    '''
    This function takes in the DataFrame from get_telco_churn_data
    and returns the DataFrame with preprocessing applied 
    '''
    # Drop duplicated columns and rows
    df = df.loc[:, ~df.columns.duplicated()]
    df.drop_duplicates(inplace=True)

    # Replace ' ' in 'total_charges' column with '0' and change its data type to float
    df.total_charges = df.total_charges.replace(' ', '0').astype(float)

    # Change data type for boolean columns
    for col in ['churn', 'partner', 'dependents', 'phone_service']:
        df[col] = df[col].map({'No': 0, 'Yes': 1})

    return df

In [ ]:
def split_data(df):
    '''
    This function takes in a DataFrame and returns train, validate, and test DataFrames.
    '''
    # Create train_validate and test datasets
    train_validate, test = train_test_split(df, test_size=.2, random_state=123, stratify=df.churn)

    # Split train_validate into train and validate datasets
    train, validate = train_test_split(train_validate, test_size=.3, random_state=123, stratify=train_validate.churn)

    return train, validate, test


In [ ]:
# To inspect the first few rows of the DataFrame, you can use the head functiotelco.head()
telco.head()

   

_________________

## Explore 

In [ ]:
# Change the name of the data to 'telco'
telco = prep_telco_churn(telco)

In [ ]:
prep_telco_churn(telco)

In [ ]:
# Review the data
telco.head()

In [ ]:
#The data types in this dataset are integers and object.
telco.info()

### Data Summary
- 13 object data types
- 9 integer data types
- 2 float data type
- 0 null values


In [ ]:
# Summary of the statistical properties of telco dataset
telco.describe()

_______

### Probe the data for insights.


In [ ]:
#We need to understand why customers are churning
telco.churn.value_counts()

# 26.54% (1 in 3)of customers churn, totaling 1869 out of 7043 


In [ ]:
# Determine how many are senior citizens and non-senior citizens 

telco.senior_citizen.value_counts()

# 16.21% (1 in 6) of customers are seniors (1142 out of 7043)

In [ ]:
# Average tenure, max tenure, min tenure
telco.tenure.mean(), telco.tenure.max(), telco.tenure.min()

In [ ]:
# Average monthly charges, max monthly charges, min monthly charges
telco.monthly_charges.mean(), telco.monthly_charges.max(), telco.monthly_charges.min()

In [ ]:
# Group by churn and senior_citizen 
# Senior_churn= telco.groupby(['churn','senior_citizen'])

_________

In [ ]:
# Value counts for churn as int not obj
(telco.churn == 'Yes').astype(int).value_counts()

In [ ]:
telco.head()

In [ ]:
telco.head()

In [ ]:
# Show all column names for next steps of value count loop
telco.columns

In [ ]:
# List the columns that will be used in the loop.

columns = ['gender', 'senior_citizen', 'partner', 'dependents', 'tenure',
       'phone_service', 'multiple_lines', 'internet_service_type_id',
       'online_security', 'online_backup', 'device_protection', 'tech_support',
       'streaming_tv', 'streaming_movies', 'contract_type_id',
       'paperless_billing', 'payment_type_id', 'monthly_charges',
       'total_charges', 'churn', 'contract_type_id', 'contract_type',
       'payment_type_id', 'payment_type', 'internet_service_type_id',
       'internet_service_type']

In [ ]:
# Create for loop print value counts and percents
for col in columns:
    print(col)
    print(telco[col].value_counts())
    print('----------------------------------------')
    print(telco[col].value_counts(normalize=True))
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

### Data Prep

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
# Identify a  correlation, age appears to correlate with churn the most
telco_correlation = telco.corr()
telco_correlation
# The highest positive correlation with churn are senior_citizen and monthly_charges 

In [ ]:
# Provides correlation with JUST churn
telco_corr_churn = telco_correlation['churn']
telco_corr_churn

# senior_citizen is still the highest correlation
#This data will help us pinpoint why customers churn


In [ ]:
# Plot displays senior citizen (1) vs non-senior citizen (0)
# Plot displays churn (1) vs do not churn (0)
sns.countplot(x='churn', hue='senior_citizen', data=telco)


In [ ]:
# Heatmap visual to show data correlation 
plt.figure(figsize=(16,9))

sns.heatmap(telco.corr(), cmap='YlOrRd', center=0, annot=True)

plt.title('Correlation of Telco Data')

plt.show()

_______

## Modeling/ Testing

### Finding the best  model 
- churn and senior_citizen are both categorical
- 2 discrete variables
- use chi2 testing

In [ ]:
# Create confusion matrix
observed = pd.crosstab(telco.churn, telco.senior_citizen)
observed

## Hypothosis:

- **$H_{0}$**: there is no relationship between churn and customer age
<br>

- **$H_{a}$**: there is a relationshp between churn and customer age
<br>

- **True Positive**: Correctly predict there is a relationship and there is a relationship
<br>

- **True Negative**: Correctly Predict there is no relationship and there is not relationship
<br>

- **False Positive**: Incorectly predict there is a relationship and there is no relationship
<br>

- **False Negative**: Incorrectly predict there is no relationship and there is a relationship

In [ ]:
# Define alpha
alpha = 0.05

In [ ]:
# chi2 contingency returns 4 different values
chi2, p, degf, expected = stats.chi2_contingency(observed)
chi2, p, degf, expected

In [ ]:
# Streamline data
print('Observed\n')
print(observed.values)
print('---------------------\nExpected\n')
print(expected.astype(int))
print('---------------------\n')
print(f'chi^2 = {chi2:.4f}')
print(f'p     = {p:.4f}')

In [ ]:
if p< alpha:
    print('We reject the null hypothesis')
else:
    print('We fail to reject the null hypothesis')

### We reject $H_0$. We believe that there is a relationship between churn and customer age

_________________________________

### Train Split

In [ ]:
import prepare
import acquire

In [ ]:
# Create train,validate and test datasets

train, test = train_test_split(telco, test_size=.2, random_state=123)
train, validate = train_test_split(train, test_size=.3, random_state=123)

In [ ]:
# Explore and plot with only train set

In [ ]:
# Crosstab, confusion matrix

In [ ]:
pd.crosstab(train.churn, train.model5)

In [ ]:
# make baseline

In [ ]:
X_col= ['senior_citizen','tenure', 'internet_service_type_id', 'contract_type_id']
y_col= 'churn'

In [ ]:
X_train = train[X_col]
y_train= train[y_col]

X_validate = validate[X_col]
y_validate= validate[y_col]

X_test = test[X_col]
y_test= test[y_col]